In [16]:
import pandas as pd
import numpy as np
import xgboost as xgb
import os
import matplotlib.pyplot as plt

from math import sqrt
import datetime
import pickle

import sklearn
from sklearn import preprocessing

# Load saved models and associated data

In [2]:
fs_lm_lags_1_12 = pickle.load(open('fs_lm_lags_1_12.pkl', 'rb'))
fs_lm_lags_1_12_predictor_cols = pickle.load(open('fs_lm_lags_1_12_predictor_cols.pkl', 'rb'))

fs_xgb_lags_1_12   = pickle.load(open('fs_xgb_lags_1_12.pkl', 'rb'))
fs_xgb_lags_1_to_6 = pickle.load(open('fs_xgb_lags_1_to_6.pkl', 'rb'))

stacking_fs_xgb = pickle.load(open('stacking_fs_xgb.pkl', 'rb'))

# Make predictions for each model, using appropriately pre-processed data

## xgboost using lags 1 and 12

In [3]:
# load (mostly pre-processed) data -- i.e. without model-specific pre-processing
X = pickle.load(open('fs_data_lags_1_12.pkl', 'rb')).query("date_block_num==34")




In [5]:
# make predictions
fs_xgb_lags_1_12_preds = fs_xgb_lags_1_12.predict(xgb.DMatrix(X[fs_xgb_lags_1_12.feature_names].values,
                                                 feature_names = fs_xgb_lags_1_12.feature_names),
                                                 ntree_limit   = fs_xgb_lags_1_12.best_iteration)

## xgboost using lags 1 to 6

In [ ]:
# load (mostly pre-processed) data -- i.e. without model-specific pre-processing
X = pickle.load(open('fs_data_lags_1_to_6.pkl', 'rb')).query("date_block_num==34")


In [ ]:
# make predictions
fs_xgb_lags_1_to_6_preds = fs_xgb_lags_1_12.predict(xgb.DMatrix(X[fs_xgb_lags_1_to_6.feature_names].values,
                                                 feature_names = fs_xgb_lags_1_to_6.feature_names),
                                                 ntree_limit   = fs_xgb_lags_1_to_6.best_iteration)

## linear model using lags 1 and 12

In [22]:
def category_encoding(fs_data):

    item_category_encoder             = preprocessing.OneHotEncoder(sparse=False)
    encoded_item_category_id          = pd.DataFrame(data=item_category_encoder.fit_transform(fs_data[['item_category_id']])) 
    encoded_item_category_id.columns  = [str(col) + '_item_category' for col in encoded_item_category_id.columns]
    
    shop_encoder            = preprocessing.OneHotEncoder(sparse=False)
    encoded_shop_id         = pd.DataFrame(data=shop_encoder.fit_transform(fs_data[['shop_id']]))
    encoded_shop_id.columns = [str(col) + '_shop' for col in encoded_shop_id.columns]
    
    # join the encoded versions to the main df
    fs_data = pd.concat([fs_data.reset_index(), encoded_item_category_id, encoded_shop_id], axis=1)

    return(fs_data.reset_index())

X = pickle.load(open('../data/fs_data_lags_1_12.pkl', 'rb'))
X = category_encoding(X) # original encoding uses all the data, so do it again here ...
X = X.query("date_block_num==34") # ...and then filter to test data only

In [27]:
fs_lm_lags_1_12_preds = fs_lm_lags_1_12.predict(X[fs_lm_lags_1_12_predictor_cols])

# Run stacked model

In [29]:
# put all the level 1 predictions into a data frame, remembering to clip them
level_1_predictions = pd.DataFrame({
    'fs_xgb_lags_1_12_preds'   : np.clip(0, fs_xgb_lags_1_12_preds, 20),
    'fs_xgb_lags_1_to_6_preds' : np.clip(0, fs_xgb_lags_1_to_6_preds, 20),
    'fs_lm_lags_1_12_preds'    : np.clip(0, fs_lm_lags_1_12_preds, 20)
 })

In [44]:
# make predictions for the stacked model
preds  = stacking_fs_xgb.predict(xgb.DMatrix(level_1_predictions[stacking_fs_xgb.feature_names],
                                             feature_names=stacking_fs_xgb.feature_names),
                                 ntree_limit=stacking_fs_xgb.best_iteration)



In [45]:
preds = np.clip(preds, 0, 20)

# Create submission file

In [46]:
# create submission file
submission_filename = 'future_sales_' + datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + '.csv'

submission = pd.DataFrame({
  'ID' : [int(x) for x in X.ID],
  'item_cnt_month' : preds
 })

# make sure this is in the same order as the test data
submission = submission.sort_values('ID')

In [ ]:
submission.to_csv('../submissions/' + submission_filename,index=False)